#**Reconhecimento de Padrões**

*Trabalho 4: PCA e Classificador SVM*

*- Kernel Linear*

*- Renoir Sampaio (389113)*

##**Preparação dos dados e informações**


In [ ]:
# Importando bibliotecas
import numpy as np # Algebra Linear
from numpy.linalg import inv, det, norm # Inversa e determinante
import pandas as pd # Processamento de dados
import seaborn as sns # Plotagem
import matplotlib.pyplot as plt # Plotagem
import math # Matemática
from random import randint, uniform # Aleatoriedade
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import classification_report, confusion_matrix
!pip install qpsolvers
from qpsolvers import solve_qp

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Realiza a leitura do dataset
df = pd.read_csv('/content/TEJ - Normalized_2013_0430_Taiwan_data.csv', sep=',', encoding='latin-1', header=None)

In [ ]:
# Renomeando coluna de classes
df = df.drop([0])
df = df.rename(columns = {0: 'classe'}, inplace = False)

In [ ]:
# Selecionando atributos que serão usados
df = df.iloc[0:1000, 0:94]

In [ ]:
# Tornando dados do tipo float
df = df.astype(float)

In [ ]:
# Infomações sobre a base de dados
print(f'Número de amostras da base de dados: {df.shape[0]}')
print(f'Número de atributos da base de dados: {df.shape[1]-1}')
vetor_classes = np.unique(df['classe'])
print(f'Classes presentes na base de dados: {vetor_classes.tolist()}')

Número de amostras da base de dados: 1000
Número de atributos da base de dados: 93
Classes presentes na base de dados: [0.0, 1.0]


Nota: Esta base de dados foram coletados do Taiwan Economic Journal para os anos de 1999 a 2009. A falência da empresa foi definida com base nos regulamentos de negócios da Bolsa de Valores de Taiwan.

In [ ]:
# p Atributos (linhas) e N amostras (colunas)
df = df.T

In [ ]:
# Mostrando o dataset
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000
classe,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.370594,0.464291,0.426071,0.399844,0.465022,0.388680,0.390923,0.508361,0.488519,0.495686,0.482475,0.444401,0.491152,0.474041,0.506703,0.513821,0.488909,0.535953,0.504071,0.487398,0.485253,0.504558,0.512017,0.494857,0.509969,0.444986,0.519280,0.547409,0.500853,0.416126,0.462195,0.476088,0.505874,0.486374,0.494369,0.475162,0.462926,0.478916,0.481061,0.486228,...,0.522595,0.519524,0.503486,0.502218,0.494808,0.476186,0.532199,0.507824,0.581046,0.517379,0.511481,0.518159,0.507727,0.517623,0.415883,0.495052,0.573490,0.514259,0.520938,0.542583,0.505046,0.484035,0.514600,0.505241,0.513333,0.573929,0.569736,0.558816,0.530737,0.518500,0.541559,0.584410,0.448301,0.519085,0.577097,0.507337,0.460537,0.494418,0.476186,0.512309
2,0.424389,0.538214,0.499019,0.451265,0.538432,0.415177,0.445704,0.570922,0.545137,0.550916,0.567543,0.549717,0.551570,0.533308,0.575829,0.571086,0.560238,0.590438,0.559802,0.543720,0.545573,0.564490,0.563672,0.548136,0.561382,0.503652,0.563618,0.593055,0.563945,0.470235,0.536034,0.544483,0.570704,0.544756,0.550916,0.564544,0.516354,0.541321,0.539468,0.548027,...,0.563345,0.556967,0.546500,0.541758,0.557894,0.519734,0.574357,0.577082,0.628543,0.578227,0.567433,0.578990,0.559474,0.594908,0.468164,0.557512,0.622874,0.563399,0.589130,0.574520,0.567979,0.541594,0.570159,0.547100,0.557348,0.621893,0.619767,0.603576,0.571686,0.555877,0.601068,0.632577,0.504634,0.572830,0.609409,0.564217,0.521369,0.553969,0.539686,0.579427
3,0.405750,0.516730,0.472295,0.457733,0.522298,0.419134,0.436158,0.559077,0.543284,0.542963,0.538198,0.498956,0.543391,0.523690,0.569838,0.558756,0.540286,0.580920,0.558649,0.533647,0.534665,0.553027,0.569035,0.540446,0.552599,0.495530,0.568606,0.592484,0.550565,0.463783,0.514428,0.529686,0.559827,0.540500,0.548584,0.525028,0.517105,0.529311,0.530864,0.544140,...,0.568392,0.571819,0.549441,0.557953,0.549548,0.511269,0.579795,0.557792,0.615343,0.560041,0.556454,0.561700,0.549441,0.580599,0.460678,0.542909,0.608384,0.562771,0.574763,0.597409,0.543337,0.543284,0.565448,0.538198,0.557685,0.604047,0.605653,0.592484,0.570587,0.572943,0.595696,0.624712,0.494405,0.567696,0.605011,0.555490,0.505969,0.544087,0.526527,0.566358
4,0.601457,0.610235,0.601450,0.583541,0.598783,0.590171,0.619950,0.601738,0.603612,0.599209,0.614026,0.623712,0.608131,0.600578,0.604686,0.621773,0.606524,0.618451,0.598344,0.636259,0.622177,0.607446,0.618934,0.609507,0.602956,0.611763,0.626544,0.609738,0.602027,0.599115,0.599987,0.618869,0.601169,0.603475,0.599108,0.614278,0.622703,0.607042,0.600881,0.598445,...,0.602999,0.607266,0.602531,0.603864,0.599036,0.604484,0.617550,0.606199,0.614451,0.604282,0.614883,0.600996,0.617233,0.604542,0.598870,0.601724,0.613348,0.601327,0.602891,0.616080,0.601796,0.612188,0.612116,0.614163,0.604362,0.605248,0.615647,0.605522,0.613500,0.606372,0.621737,0.609904,0.597839,0.606120,0.619726,0.603259,0.600045,0.607713,0.607742,0.612296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
# Divisão treino e teste
def divTesteTreinoTransp(df, percent, lengh):
  i = lengh*percent/100
  return df[0:int(i)], df[int(i):lengh]

In [ ]:
# Criação de função para cálculo de acurácia baseado na coluna 'classe'  
# da tabela dados gerais, onde estão localizados os labels
def getAccuracy(testSet, predictions, tamanho):
    correct = 0
    for x in range(len(testSet)):
        if (testSet[x] == predictions[x]):
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [ ]:
# Criar dataset aleatório
df_sample = df.T.sample(frac=1)
df_sample = df_sample.T

In [ ]:
# Divisão treino e teste 70% e 30%
treino, teste = divTesteTreinoTransp(df_sample.T, 70, len(df.T))
treino = treino.T
teste = teste.T

In [ ]:
# Mostrando dados de treino
treino

,505,117,497,671,382,758,620,808,583,892,144,233,746,453,743,619,230,202,981,814,961,53,403,701,644,630,551,276,767,133,258,635,3,540,555,625,312,21,99,439,...,575,199,279,962,377,304,846,239,973,657,88,676,523,335,395,780,932,257,350,375,16,513,11,488,556,111,146,925,84,639,684,913,354,112,941,793,317,818,972,114
classe,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
1,0.479988,0.485595,0.489251,0.504656,0.471993,0.530395,0.427144,0.516014,0.484668,0.506557,0.492858,0.345781,0.504266,0.470872,0.481402,0.474480,0.524692,0.469020,0.505046,0.534442,0.522595,0.521913,0.511237,0.563106,0.514600,0.561010,0.428752,0.491786,0.507678,0.470141,0.398089,0.480720,0.426071,0.473943,0.428460,0.499829,0.473358,0.485253,0.526885,0.506411,...,0.467508,0.468093,0.440647,0.519524,0.543460,0.557939,0.532882,0.467508,0.507727,0.454980,0.477697,0.577390,0.512066,0.484376,0.494272,0.493053,0.565690,0.435041,0.481012,0.485984,0.513821,0.488276,0.482475,0.493053,0.456101,0.438746,0.479891,0.510554,0.480086,0.516453,0.494418,0.533272,0.494613,0.503242,0.483596,0.480183,0.465363,0.477405,0.518159,0.479355
2,0.540340,0.570377,0.550698,0.559256,0.546119,0.580353,0.488007,0.545901,0.530473,0.582370,0.551243,0.393153,0.559802,0.513901,0.543666,0.539631,0.572721,0.528729,0.567979,0.591201,0.563345,0.564871,0.564762,0.602213,0.559638,0.609137,0.470617,0.555931,0.552715,0.520388,0.448103,0.545301,0.499019,0.537724,0.475796,0.560619,0.517608,0.545573,0.571522,0.570214,...,0.512920,0.525240,0.497601,0.556967,0.571195,0.624618,0.601123,0.512538,0.559474,0.471925,0.541703,0.619440,0.550262,0.531618,0.523005,0.554350,0.611644,0.492205,0.528838,0.551025,0.571086,0.548136,0.567543,0.590711,0.506705,0.504525,0.535434,0.567597,0.541594,0.567597,0.548626,0.592673,0.539850,0.549171,0.551243,0.545028,0.530255,0.542085,0.578990,0.534834
3,0.535789,0.541678,0.531667,0.556829,0.526955,0.595321,0.479308,0.567375,0.539215,0.549601,0.540661,0.386584,0.551903,0.514482,0.531024,0.520049,0.567696,0.521441,0.543337,0.576262,0.568392,0.568178,0.564912,0.606510,0.570159,0.615504,0.475561,0.543498,0.556775,0.522030,0.447294,0.536378,0.472295,0.526259,0.470743,0.556133,0.529472,0.534665,0.571819,0.556668,...,0.518443,0.519246,0.493870,0.571819,0.592162,0.605653,0.578992,0.518979,0.549441,0.513250,0.532309,0.614969,0.555115,0.536967,0.536538,0.542588,0.607313,0.492585,0.535093,0.536110,0.558756,0.539001,0.538198,0.546496,0.502382,0.497511,0.533862,0.553616,0.534611,0.558220,0.557203,0.591199,0.547406,0.559505,0.535628,0.531345,0.518229,0.530864,0.561700,0.532951
4,0.602855,0.603237,0.597220,0.611150,0.596629,0.613046,0.609421,0.606934,0.605543,0.604794,0.601594,0.586618,0.609377,0.591843,0.605125,0.614220,0.608736,0.603360,0.601796,0.608282,0.602999,0.609291,0.600066,0.614019,0.611986,0.615410,0.589400,0.600513,0.609305,0.597306,0.589465,0.615136,0.601450,0.600607,0.594106,0.613838,0.596758,0.622177,0.619575,0.601673,...,0.592132,0.593566,0.591108,0.607266,0.600967,0.613702,0.609392,0.593133,0.617233,0.597198,0.619863,0.604859,0.619179,0.608275,0.597710,0.611712,0.613925,0.596715,0.596744,0.605219,0.621773,0.598041,0.614026,0.600232,0.599187,0.595663,0.598142,0.605839,0.597904,0.611064,0.608527,0.620534,0.601378,0.603893,0.624223,0.600902,0.603043,0.605025,0.600996,0.598574
...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
# Mostrando dados de teste
teste

,715,622,91,477,473,638,606,861,529,885,665,64,226,884,610,471,994,562,273,990,299,928,545,427,904,70,765,558,231,613,306,286,823,186,908,527,775,790,120,201,...,104,80,624,212,259,549,38,94,807,856,277,433,270,332,649,103,725,261,843,740,221,735,959,372,560,920,193,376,726,174,975,406,481,422,154,955,965,301,4,36
classe,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
1,0.526300,0.514405,0.476673,0.446887,0.537074,0.490372,0.437430,0.578560,0.486228,0.503924,0.641739,0.482280,0.524692,0.501097,0.485595,0.493248,0.519085,0.399844,0.522644,0.518500,0.449374,0.495149,0.469556,0.462000,0.479647,0.496368,0.495588,0.453127,0.447229,0.605226,0.370643,0.493589,0.517330,0.468142,0.522595,0.447960,0.486082,0.555404,0.384927,0.493687,...,0.502706,0.523034,0.488276,0.514942,0.500561,0.493248,0.478916,0.488617,0.507288,0.508946,0.496612,0.450300,0.473992,0.485887,0.486569,0.538439,0.476332,0.468873,0.437284,0.506118,0.526837,0.480330,0.491542,0.506069,0.461317,0.487106,0.469702,0.441964,0.486716,0.526154,0.415883,0.502901,0.498562,0.442354,0.527617,0.498708,0.494808,0.478038,0.399844,0.475162
2,0.574193,0.575392,0.512048,0.525240,0.572503,0.563563,0.501799,0.627780,0.551734,0.546282,0.685292,0.552933,0.582479,0.538596,0.547209,0.554187,0.572830,0.441016,0.579099,0.555877,0.505179,0.547700,0.536415,0.519679,0.523932,0.549062,0.560183,0.514991,0.501635,0.621838,0.434693,0.547590,0.568360,0.549444,0.556204,0.545028,0.545083,0.608264,0.410488,0.558493,...,0.555277,0.563127,0.548735,0.575229,0.548572,0.550153,0.541321,0.542957,0.557130,0.540177,0.546337,0.492532,0.522405,0.538923,0.541321,0.583297,0.540558,0.543447,0.505342,0.566180,0.578173,0.546064,0.554950,0.559202,0.470017,0.549935,0.539468,0.488552,0.553423,0.588476,0.468164,0.547045,0.537178,0.533799,0.576047,0.565962,0.557894,0.527148,0.451265,0.564544
3,0.576316,0.569088,0.531452,0.499920,0.583918,0.543766,0.489480,0.610311,0.537877,0.556614,0.667220,0.540125,0.569409,0.556347,0.542427,0.539483,0.567696,0.440495,0.568606,0.572943,0.500348,0.548263,0.521548,0.512929,0.530007,0.545532,0.546871,0.501954,0.492639,0.638150,0.416136,0.543016,0.557632,0.517640,0.564109,0.491782,0.537877,0.591895,0.429573,0.560308,...,0.553349,0.570159,0.536753,0.560683,0.551207,0.545747,0.529311,0.542213,0.552439,0.553938,0.552278,0.501312,0.531827,0.534879,0.548477,0.582579,0.528080,0.514000,0.493656,0.556186,0.580652,0.534129,0.541946,0.565073,0.511376,0.536110,0.524118,0.494138,0.536913,0.586862,0.460678,0.559131,0.546603,0.489641,0.571765,0.547406,0.549548,0.530810,0.457733,0.525028
4,0.617197,0.612167,0.592499,0.600254,0.600455,0.605219,0.608772,0.615835,0.600239,0.602581,0.620988,0.619049,0.611727,0.606545,0.608678,0.603749,0.606120,0.591973,0.601284,0.606372,0.589840,0.612174,0.599295,0.598985,0.609666,0.612426,0.604765,0.601169,0.593501,0.614574,0.594661,0.599583,0.604513,0.602055,0.604044,0.594423,0.604873,0.611705,0.586265,0.602574,...,0.605752,0.607165,0.609118,0.605781,0.603965,0.607893,0.607042,0.609673,0.603050,0.607972,0.600333,0.590424,0.599677,0.604073,0.609594,0.622919,0.609132,0.595605,0.601018,0.616325,0.605493,0.604376,0.602668,0.599756,0.599324,0.608844,0.609060,0.594697,0.603425,0.599504,0.598870,0.602509,0.595526,0.595432,0.604953,0.606841,0.599036,0.599338,0.583541,0.614278
...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
# Retirando classe do dataset de treino
treino_noclass = treino.T.drop(columns = ['classe'])
treino_noclass = treino_noclass.T

In [ ]:
# Retirando classe do dataset de teste
teste_noclass = teste.T.drop(columns = ['classe'])
teste_noclass = teste_noclass.T

In [ ]:
# Vetor média dos atributos de treino
treino_noclass_mean = treino_noclass.T.mean()

In [ ]:
# Subtrair o valor médio de cada atributo da base de dados de treino
df_new = []
for i in range(len(treino_noclass)):
  df_prov = []
  for j in range(len(treino_noclass.T)):
    df_prov.append(treino_noclass.values[i][j] - treino_noclass_mean.values[i])
  df_new.append(df_prov)
df_new = pd.DataFrame(df_new)

In [ ]:
# Matriz COV em função dos dados de treino
matrixCov_treino = pd.DataFrame.cov(df_new.T)

In [ ]:
# Autovetores (U) autovalores (sigma)
sigma, U = np.linalg.eig(matrixCov_treino)

In [ ]:
# Transformando 'U' em formato DataFrame
U = pd.DataFrame(U)

In [ ]:
# Multiplicando qualquer coluna por ela mesma, valor esperado: 1
U[0].T @ U[0]

1.0000000000000002

In [ ]:
# Transformando sigma em matriz diagonal
d_sigma = np.diag(sigma)

In [ ]:
# Transformando matriz diagonal 'sigma' em formato DataFrame
d_sigma = pd.DataFrame(d_sigma)

In [ ]:
# Somatório dos autovalores
sum_sigma = sigma.sum()

In [ ]:
# Definindo q para dimenção da matriz de U transformada
# buscando 95% dos valores
for i in range(df_new.shape[0]):
  soma_prov = sigma[0:i].sum()
  porcent_sigma = soma_prov / sum_sigma * 100
  print(f'Com {i}: {int(porcent_sigma)}%')
  if porcent_sigma >= 95:
    q = i
    break
print(f'Sendo assim, a quantidade de colunas escolhidas foi {q}')

Com 0: 0%
Com 1: 50%
Com 2: 68%
Com 3: 78%
Com 4: 85%
Com 5: 88%
Com 6: 91%
Com 7: 93%
Com 8: 94%
Com 9: 95%
Sendo assim, a quantidade de colunas escolhidas foi 9


In [ ]:
# Criar matriz transformada, valores descorrelacionados
matrix_transform = U.T @ df_new

In [ ]:
# Matriz COV por multiplicação matricial com autovetores
# e dados de treino transformados
matrixCov_treino_transform = ((U @ matrixCov_treino) @ U.T)

In [ ]:
# Matriz identidade por serem vetores ortogonais
matrizIdentidade = U.T @ U

In [ ]:
# Decompondo matriz de teste
df_decomp = ((U @ d_sigma) @ U.T)

In [ ]:
# Nova matriz de tranformação (que antes era o próprio U) será de dimenção pxq
# onde p ainda sera o número de atributos e q < p (definido previamente). 
# U de dimenção pxq transposto que multiplica df_new de dimenção pxN.
U_transf = U.iloc[:, 0:q]
matrix_transform_new = U_transf.T @ df_new

In [ ]:
# Construindo o modelo para cada novo vetor dos dados de teste
df_teste = []
for i in range(len(teste_noclass)):
  df_prov = []
  for j in range(len(teste_noclass.T)):
    df_prov.append(teste_noclass.values[i][j] - treino_noclass_mean.values[i])
  df_teste.append(df_prov)
df_teste = pd.DataFrame(df_teste)
matrix_transform_teste = U_transf.T @ df_teste

## ***SVM (Linear):***

In [ ]:
# 15 interações
all_accuracy_treino = []
all_accuracy_teste = []
melhor_acuracia = 0
melhor_C = 0
for p in range(15):
  print('#####################################################################')
  print(f'Interação: {p+1}/15')
  # Ordem do Kernel polinomial
  d = randint(30, 100)
  print(f'Valor de d = {d}')

  # Matriz X do treino
  X = matrix_transform_new

  # Vetor com as classes do treino
  treino_sample_class0 = treino.T.loc[treino.T['classe'] == 0]
  treino_sample_class1 = treino.T.loc[treino.T['classe'] == 1]
  y0 = -np.ones(treino_sample_class0.shape[0])
  y0 = pd.DataFrame(y0)
  y1 = np.ones(treino_sample_class1.shape[0])
  y1 = pd.DataFrame(y1)
  y = pd.concat([y0,y1])
  y_reset_index = y.reset_index(drop=True)

  # Cálculo do Kernel polinomial
  K = ((X.T @ X + 1) ** d) # Kernel linear

  # Formulação da programação quadrática
  C = uniform(0.001, 1000) # Constante de relaxamento
  a = np.zeros((X.shape[1], X.shape[1]), int)
  np.fill_diagonal(a, y_reset_index)
  y_diag = pd.DataFrame(a)
  P = ((y_diag * K) * y_diag)
  P = (P + P.T)/2
  P = np.array(P)
  P += np.identity(X.shape[1]) * 0.001
  P = P.astype('double')
  q = -C*np.ones(X.shape[1])
  G = -np.identity(X.shape[1])
  G = G.astype('double')
  h = np.zeros(X.shape[1])
  A = np.array(y_reset_index.T)[0]
  beq = np.array([0])

  # Chamada da função de otimização
  alpha = solve_qp(P, q, G, h, A, beq)

  # Achar vetores suporte
  vet_sup = np.argwhere(alpha > 1e-3)

  # Cálculo da constante b
  b = 0
  for i in vet_sup[:,0]:
    b = b + (1 / y_reset_index.iloc[i].values[0])
    for j in vet_sup[:,0]:
      b = b - (y_reset_index.iloc[j].values[0] * alpha[j] * K.iloc[j,i])
  b = b/vet_sup.shape[0]

  ############################ FASE DE TESTE ##################################
  
  # Matriz X do teste
  X = matrix_transform_teste

  # Vetor com as classes do teste
  teste_sample_class0 = teste.T.loc[teste.T['classe'] == 0]
  teste_sample_class1 = teste.T.loc[teste.T['classe'] == 1]
  y0 = -np.ones(teste_sample_class0.shape[0])
  y0 = pd.DataFrame(y0)
  y1 = np.ones(teste_sample_class1.shape[0])
  y1 = pd.DataFrame(y1)
  y = pd.concat([y0,y1])
  y_reset_index_teste = y.reset_index(drop=True)

  # Cálculo do Kernel polinomial
  K = ((X.T @ X + 1) ** d) # Kernel linear

  # Estimar classes
  aux = 0
  for j in vet_sup[:,0]:
    if j < K.shape[0]:
      aux = aux + (y_reset_index_teste.iloc[j].values[0] * alpha[j] * K.iloc[j,:])
  y_est_teste = np.sign((aux + b).T)
  y_est_teste = y_est_teste.astype('double')
  y_est_teste = pd.DataFrame(y_est_teste)

  # Vetores de índices reais para comparação
  indices_teste = y_reset_index_teste.T.values[0]
  indices_teste = indices_teste.astype(float)

  # Vetores de índices previstos para comparação
  indices_previstos_teste = y_est_teste.T.values[0]
  indices_previstos_teste = indices_previstos_teste.astype(float)
  print(f'Matriz de confusão:\n{confusion_matrix(indices_teste, indices_previstos_teste)}')

  # Teste de acurácia com labels corretos e labels previstos (classificados)
  accuracy_teste = getAccuracy(indices_teste, indices_previstos_teste, len(indices_teste))
  all_accuracy_teste.append(accuracy_teste)
  print(f'Acurácia de teste dessa interação: {int(accuracy_teste)}%')
  print('#####################################################################')
  if p < 14:
    print('|/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/_')

  # Melhor acurácia e hiperparâmetros
  if accuracy_teste > melhor_acuracia:
    melhor_acuracia = accuracy_teste
    melhor_C = C

print('\n')
# Média total das acurácias de teste
print(f'Média das acurácias de teste: {int(np.mean(all_accuracy_teste))}%')
# Melhor hiperparâmetro
print(f'No melhor valor de acurácia do teste, que foi igual a {int(melhor_acuracia)}%, o melhor valor para hiperparâmetro C foi {melhor_C}')

#####################################################################
Interação: 1/15
Valor de d = 78
Matriz de confusão:
[[273  10]
 [ 15   2]]
Acurácia de teste dessa interação: 91%
#####################################################################
|/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/_
#####################################################################
Interação: 2/15
Valor de d = 82
Matriz de confusão:
[[273  10]
 [ 15   2]]
Acurácia de teste dessa interação: 91%
#####################################################################
|/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/_
#####################################################################
Interação: 3/15
Valor de d = 60
Matriz de confusão:
[[276   7]
 [ 15   2]]
Acurácia de teste dessa interação: 92%
#####################################################################
|/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/_
############################

Nota: Podemos notar que, para esse problema, o kernel do tipo linear nos retorna uma boa acurácia, porém não consegue ter um bom resultado de classificação, como vemos no baixo valor de recall e a baixa precisão. 